## About IMDB Dataset

* データセット: 映画のレビューコメントを集めた大規模なテキストデータ
* レビューにはポジティブかネガティブかのラベルがついている
    - 実際のラベル対応については以下も参考になる
    - https://note.com/e_dao/n/ne73960e21b79
* 25,000件の訓練データ、25,000件のテストデータが含まれる

## Import Libraries and Set Device

In [18]:
from google.colab import drive
drive.mount('/content/drive/')

from google.colab import drive
drive.mount('/content/drive/')

import os, sys
print('Current Directory: ', os.getcwd())

ROOT_PATH = '/content/drive/My Drive/Colab Notebooks/'
sys.path.append(ROOT_PATH)

Mounted at /content/drive/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Current Directory:  /content


In [1]:
!pip install portalocker==2.3.0
!pip install torch==1.13.1
!pip install torchdata==0.6.0
!pip install torchtext==0.15.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset , RandomSampler, SequentialSampler
from torchtext.datasets import IMDB
from torchtext.data.functional import to_map_style_dataset
from torch.optim import AdamW

import transformers
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import AdamW
from transformers import AutoModel, BertTokenizerFast

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
import random



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("The device is : ", device)

The device is :  cuda


## Encode & Create Dataset

In [25]:
# データセットをロードしてマップスタイルに変換
train_iter, test_iter = IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# トークナイザの初期化
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Positiveの場合(2→1)に変換する
# Negativeの場合(1→0)に変換する
def adjust_labels(labels):
    labels = labels - 1
    return labels

# サンプリング
train_num = 1024
test_num = 128
train_sampled = random.sample(list(train_dataset), train_num)
test_sampled = random.sample(list(test_dataset), test_num)

# サンプリングされたデータセットからテキストを抽出（エンコード時にreturn tensorが指定できるので今はテンソル化不要）
train_texts = [text for (_, text) in train_sampled]
test_texts = [text for (_, text) in test_sampled]

# サンプリングされたデータセットから整数ラベルを抽出
train_labels = torch.tensor([label for (label, _) in train_sampled], dtype=torch.long)
test_labels = torch.tensor([label for (label, _) in test_sampled], dtype=torch.long)

train_labels = adjust_labels(train_labels)
test_labels = adjust_labels(test_labels)


# データのエンコード（トークン化）
encoded_train = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
encoded_test = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

# エンコードされたデータからTensorDatasetを作成
train_dataset = TensorDataset(encoded_train['input_ids'], encoded_train['attention_mask'], train_labels)
test_dataset = TensorDataset(encoded_test['input_ids'], encoded_test['attention_mask'], test_labels)

# DataLoaderの設定
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # Testはシャッフルしない

print(f"Train DataLoader has {len(train_loader)} batches.")
print(f"Test DataLoader has {len(test_loader)} batches.")

print("Train labels unique values:", torch.unique(train_labels))
print("Test labels unique values:", torch.unique(test_labels))

Train DataLoader has 128 batches.
Test DataLoader has 16 batches.
Train labels unique values: tensor([0, 1])
Test labels unique values: tensor([0, 1])


## Demonstration

In [4]:
# インスタンス化
bert = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# デモ用のテキスト
texts = ["I like spyderman", "I dislike this movie"]

# テキストのバッチをエンコード
input_ids = tokenizer.batch_encode_plus(
    texts,  # テキストのリストを直接渡す
    padding=True,  # 全てのシーケンスが同じ長さになるようにパディング
    return_token_type_ids=True,  # トークンタイプIDを返す
    truncation=True,  # 最大長を超えるトークンを切り捨てる
    return_tensors="pt"  # PyTorchのテンソルとして結果を返す
)

print(input_ids)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  1045,  2066,  8645,  4063,  2386,   102],
        [  101,  1045, 18959,  2023,  3185,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


## Modeling

In [16]:
#GPUのキャッシュクリア
import gc
gc.collect()  # ガベージコレクションの強制実行
torch.cuda.empty_cache()  # CUDAキャッシュのクリア

In [17]:
# モデルの初期化（感情分析タスクのための2つのラベル）
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# ハイパーパラメータの設定 （損失関数はモデル内に含まれてるので不要）
# インスタンス化することでclassifier.biasとclassifier.weightを初期化
# AdamWはtransformersでなくpytorchのものを使用
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 10



# トレーニングモードに設定
model.train()

for epoch in range(epochs):

    print("Epoch: ", epoch, "/", epochs)
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch[0].to(model.device) # train_loaderには[input_ids, attention_mask, labels]が入ってる
        attention_mask = batch[1].to(model.device)
        labels = batch[2].to(model.device)

        #オプティマイザの初期化
        optimizer.zero_grad()

        # 順伝播計算
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        #逆伝播計算
        loss.backward()

        #重みの更新
        optimizer.step()

    print('Loss: ', total_loss)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch:  0 / 10
Loss:  73.35677639394999
Epoch:  1 / 10
Loss:  41.124380711466074
Epoch:  2 / 10
Loss:  26.27963793091476
Epoch:  3 / 10
Loss:  16.638293244875968
Epoch:  4 / 10
Loss:  13.831583279184997
Epoch:  5 / 10
Loss:  7.243148343171924
Epoch:  6 / 10
Loss:  1.819473164039664
Epoch:  7 / 10
Loss:  6.886432120751124
Epoch:  8 / 10
Loss:  1.6592801148653962
Epoch:  9 / 10
Loss:  3.7272749455296434


In [21]:
#パラメータの保存
MODEL_SAVE_PATH = "model_weights_bert.pth"  # 保存先のファイルパス
torch.save(model.state_dict(), MODEL_SAVE_PATH)

'''
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 新しいモデルインスタンスの作成
model.load_state_dict(torch.load(model_save_path))  # 保存したパラメータの読み込み
model.eval()  # 評価モードに設定
'''

'\nmodel = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 新しいモデルインスタンスの作成\nmodel.load_state_dict(torch.load(model_save_path))  # 保存したパラメータの読み込み\nmodel.eval()  # 評価モードに設定\n'

## Validation

In [23]:
def Evaluate(model, dataloader):
    model.eval() # モデルを評価モードに設定
    predictions = []
    true_labels = []

    for batch in dataloader:
        input_ids = batch[0].to(model.device)
        attention_mask = batch[1].to(model.device)
        labels = batch[2].to(model.device)

        with torch.no_grad(): # 勾配計算を無効化
            outputs = model(input_ids, attention_mask=attention_mask) # labelは予測には不要


        logits = outputs.logits # logits: 各クラスに対する生の予測値
        logits = logits.detach().cpu().numpy() #logitsテンソルを計算グラフから分離し, 勾配計算から除外
        label_ids = labels.to('cpu').numpy() #CPUに移動させることでnumpyが使える

        # 予測値と対応する正解ラベルを保存
        predictions.append(logits)
        true_labels.append(label_ids)

    # 予測値を評価
    predictions = np.concatenate(predictions, axis=0) #predictionsリストに保存された全てのバッチの予測値を1つのNumPy配列に結合
    true_labels = np.concatenate(true_labels, axis=0)#true_labelsリストに保存された全てのバッチの真のラベルを1つのNumPy配列に結合
    preds_flat = np.argmax(predictions, axis=1).flatten()  # 予測値（ロジット）から、最も高い値を持つクラスのインデックス（予測されたラベル）を選択し、配列を1次元に平坦化（flatten）することで各入力に対する最終的なクラス予測が得る
    labels_flat = true_labels.flatten() # 真のラベルの配列を1次元に平坦化することで、予測値と同じ形式で真のラベルを扱うことができ、比較が容易になる



    # 正解率を計算
    accuracy = accuracy_score(labels_flat, preds_flat)
    return accuracy

# テストデータセット上でモデルを評価
accuracy = Evaluate(model, test_loader)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 87.50%


## Use Model for Another Task

In [27]:
from transformers import BertTokenizer

# トークナイザの初期化
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# オリジナルのテキスト
text = "This is a great movie!"

# テキストをモデルの入力形式に変換
inputs = tokenizer(
    text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
    )

# 評価モードに設定
model.eval()

# GPUに移動
inputs = {key: value.to(device) for key, value in inputs.items()}

# 推論
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1).item()

# 結果
if predictions == 1:
    print("Positive review detected!")
else:
    print("Negative review detected!")

## Optional : Original Model

In [ ]:
'''
import torch.nn as nn

class BERTBinaryClassification(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, 2)
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        x = self.fc1(outputs.pooler_output) #
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


model = BERTBinaryClassification(bert)
model = model.to(device)

#省略
'''

## Fin